<a href="https://colab.research.google.com/github/futureCodersSE/python-cyber/blob/main/Automated_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create a set of automated tests from your testing plan
---

The code has been copied here again for you.

The objective of this exercise is to write code that will run the tests and report any problems.

There are a number of test frameworks available.   Unit testing frameworks are designed to help to create tests for individual functions and can be used to write black box or white box tests.

---

The code from the last worksheet has been copied here so that you can create the original database again and populate it with the first 10 records as before.

Run the two code cells below, then check that there is a file called savings.sqlite in the file panel on the left.

### Run this code cell to set up the database with an initial set of test records
---

In [ ]:
import sqlite3

# Create and populate the database ready for testing
def setup_database(db_name):
  conn = sqlite3.connect(db_name)
  cur = conn.cursor()
  query = "DROP TABLE IF EXISTS savings"
  cur.execute(query)
  conn.commit()
  query = "CREATE TABLE IF NOT EXISTS savings(user_id INTEGER PRIMARY KEY, balance REAL)"
  cur.execute(query)
  conn.commit()
  starting_balances = [44,5,42,32,25,5,26,39,45,29]
  for id in range(1, 11):
      add_new_user(id, conn)
      add_funds(id, starting_balances[id-1], conn)
  print("Database connected, savings table created and populated with testing records")
  return conn

### Run this cell to activate the app's functions
---


In [ ]:
import sqlite3
from tabulate import tabulate

# gets a copy of the data from the savings table
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def get_data(conn):
  cur = conn.cursor()
  query = "SELECT * FROM savings"
  cur.execute(query)
  data = cur.fetchall()
  columns = tuple(map(lambda x: x[0], cur.description))
  return data, columns

# prints the balance for.   The table will have a number of rows corresponding to the number of users plus a header row.
# it will be possible to get a list of users from this table
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def get_user_data(user_id, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"SELECT balance FROM savings WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchone()
    return data[0]
  else:
    return "User does not exist"

# check that a user with a given id does or doesn't exist in the savings table
# returns True or False
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def does_exist(user_id, conn):
  cur = conn.cursor()
  query = f"SELECT * FROM savings WHERE user_id = {str(user_id)}"
  cur.execute(query)
  data = cur.fetchone()
  return data is not None

# adds a user with the given id to the savings table
# returns a message to confirm that this has happened
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def add_new_user(user_id, conn):
  cur = conn.cursor()
  query = f"INSERT INTO savings(user_id,balance) VALUES({str(user_id)},0)"
  cur.execute(query)
  conn.commit()
  return True

# removes a user with the given id from the savings table after checking that the user does exist
# returns a message to confirm that this has happened or that the user doesn't exist
# ----------------------------------------------------------------------------------------------------------------------------------
def remove_user(user_id, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"DELETE FROM savings WHERE user_id={str(user_id)}"
    cur.execute(query)
    return True
  else:
    return False

# updates the balance for the given user by adding the given amount onto the balance
# returns a message to confirm that this has happened or that the user doesn't exist
# ----------------------------------------------------------------------------------------------------------------------------------
def add_funds(user_id, amount, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"UPDATE savings SET balance=balance+{str(amount)} WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchall()
    return True
  else:
    return False

# updates the balance for the given user by subtracting the given amount from the balance
# returns a message to confirm that this has happened or that the user doesn't exist
# ----------------------------------------------------------------------------------------------------------------------------------
def remove_funds(user_id, amount, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"UPDATE savings SET balance=balance - {str(amount)} WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchall()
    return True
  else:
    return False

# nullifies the connection so that is can't accidentally be used again
# ----------------------------------------------------------------------------------------------------------------------------------
def exit_app(conn):
  conn.close()
  conn = None
  print("Exiting app...")
  return conn

## Add the first three tests from the orignal manual testing table
---

These will test that the database and its table have been created correctly, with 10 rows and a range of balances.  

**Test 1** - test_get_data() Views all records in the savings table and checks that the headings are as expected and that there are 10 rows

**Test 2** - test_get_valid_user_data() runs the function get_user_data() for user_id 1 and checks that the balance is 44 as expected

**Test 3** - test_get_invalid_user_data() runs the function get_user_data() for user_id 15 and checks that it reports that this user does not exist

**YOU SHOULD NOW ADD** **Test 4** - test_get_user_data_invalid_datatype() which will runs the function get_user_data() for user_id "1" and check that it reports that the data it has been given is not of the correct type

In [ ]:
import sqlite3

# Test 1 getting data and column headings, expected user_id and balance columns and data length 10
def test_get_data():
  data, columns = get_data(db_conn)
  if columns == ("user_id", "balance") and len(data) == 10:
    print("test_get_data passed")
  else:
    print("test_get_data failed, returned {str(len(data))}")
    return 1
  return 0

# Test 2 getting a user's balance expected balance of 44 for user 1
def test_get_valid_user_data():
  user_data = get_user_data(1, db_conn)
  if user_data == 44:
    print("test_get_valid_user_data passed")
  else:
    print("test_get_valid_user_data failed, returned {str(user_data)}")
    return 1
  return 0

# test getting a user's balance for an invalid user
def test_get_invalid_user_data():
  user_data = get_user_data(15, db_conn)
  if user_data == "User does not exist":
    print("test_get_invalid_user_data passed")
  else:
    print("test_get_invalid_user_data failed, returned {str(user_data)}")
    return 1
  return 0

### Run the tests
---

*  Remove the original database  
*  Create the database and table and populate with 10 records  
*  Each test will return a 1 if it fails, add this to the number of failed tests  
*  Print the number of tests that failed.

In [ ]:
!rm /content/savings.sqlite
db_conn = setup_database("/content/savings.sqlite")
failed_tests = 0
failed_tests += test_get_data()
failed_tests += test_get_valid_user_data()
failed_tests += test_get_invalid_user_data()

print(f"{str(failed_tests)} tests failed")

### Add more tests
---
Follow the format for the first three tests.

Add a fourth test to test for an invalid data type passed to the get_user_data() function

Hint:  
```
def test_get_user_data_invalid_type():
  user_data = get_user_data("1", db_conn)
  #  add the rest
```

### and add the rest of the tests
---
from your test table